# ENTITY_TYPE

## Overview
Returns a dictionary representing an Excel entity data type, suitable for use as a custom data type in Excel. This enables the creation of structured data cards with properties and images, which can be consumed by Excel add-ins or custom connectors. The returned dictionary follows the format required by Excel's entity data type, with a top-level type, display text, and a properties dictionary. This is useful for business scenarios where you want to represent products, people, or other entities with rich metadata in Excel. The function is designed for compatibility with connectors that expect the following structure:
- `type`: always "Entity"
- `text`: display name
- `properties`: dictionary of property names to property objects (e.g., Boolean, WebImage)

## Usage
```excel
=ENTITY_TYPE([text], [discontinued], [image_url])
```

## Arguments
| Argument     | Type   | Required | Description                                 | Example |
|:------------|:-------|:---------|:--------------------------------------------|:--------|
| text        | string | No       | Display name for the entity                 | "Chai" |
| discontinued| bool   | No       | Whether the item is discontinued            | FALSE   |
| image_url   | string | No       | URL to an image representing the entity     | "https://upload.wikimedia.org/wikipedia/commons/thumb/0/04/Masala_Chai.JPG/320px-Masala_Chai.JPG" |

## Returns
| Returns | Type  | Description                                      | Example |
|:--------|:------|:-------------------------------------------------|:--------|
| Entity  | dict  | Dictionary representing an Excel entity data type | `{"type": "Entity", "text": "Chai", "properties": {"Discontinued": {"type": "Boolean", "basicValue": False}, "Image": {"type": "WebImage", "address": "https://upload.wikimedia.org/wikipedia/commons/thumb/0/04/Masala_Chai.JPG/320px-Masala_Chai.JPG"}}}` |
| Error   | string| Error message if input is invalid                 | "Error: Invalid input" |

## Examples

### Example 1: Basic entity
**Input:**
- text: "Chai"
- discontinued: FALSE
- image_url: "https://upload.wikimedia.org/wikipedia/commons/thumb/0/04/Masala_Chai.JPG/320px-Masala_Chai.JPG"

**Function call:**
```excel
=ENTITY_TYPE("Chai", FALSE, "https://upload.wikimedia.org/wikipedia/commons/thumb/0/04/Masala_Chai.JPG/320px-Masala_Chai.JPG")
```
**Output:**
| Key        | Value                                                                 |
|------------|-----------------------------------------------------------------------|
| type       | "Entity"                                                             |
| text       | "Chai"                                                               |
| properties | `{"Discontinued": {"type": "Boolean", "basicValue": False}, "Image": {"type": "WebImage", "address": "https://upload.wikimedia.org/wikipedia/commons/thumb/0/04/Masala_Chai.JPG/320px-Masala_Chai.JPG"}}` |

### Example 2: Discontinued product
**Input:**
- text: "Old Product"
- discontinued: TRUE
- image_url: "https://example.com/old_product.jpg"

**Function call:**
```excel
=ENTITY_TYPE("Old Product", TRUE, "https://example.com/old_product.jpg")
```
**Output:**
| Key        | Value                                                                 |
|------------|-----------------------------------------------------------------------|
| type       | "Entity"                                                             |
| text       | "Old Product"                                                        |
| properties | `{"Discontinued": {"type": "Boolean", "basicValue": True}, "Image": {"type": "WebImage", "address": "https://example.com/old_product.jpg"}}` |

In [ ]:
import typing

def entity_type(text: str = "Chai", discontinued: bool = False, image_url: str = "https://upload.wikimedia.org/wikipedia/commons/thumb/0/04/Masala_Chai.JPG/320px-Masala_Chai.JPG") -> typing.Union[dict, str]:
    """
    Returns a dictionary representing an Excel entity data type with the specified properties.

    Args:
        text (str, optional): Display name for the entity. Defaults to "Chai".
        discontinued (bool, optional): Whether the item is discontinued. Defaults to False.
        image_url (str, optional): URL to an image representing the entity. Defaults to a sample image.

    Returns:
        dict: Dictionary representing an Excel entity data type.
        str: Error message if input is invalid.
    """
    if not isinstance(text, str) or not text:
        return "Error: 'text' must be a non-empty string"
    if not isinstance(discontinued, bool):
        return "Error: 'discontinued' must be a boolean value"
    if not isinstance(image_url, str) or not image_url.startswith("http"):
        return "Error: 'image_url' must be a valid URL string"
    return {
        "type": "Entity",
        "text": text,
        "properties": {
            "Discontinued": {
                "type": "Boolean",
                "basicValue": discontinued
            },
            "Image": {
                "type": "WebImage",
                "address": image_url
            }
        }
    }

In [ ]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()

def test_entity_type_valid():
    result = entity_type("Chai", False, "https://upload.wikimedia.org/wikipedia/commons/thumb/0/04/Masala_Chai.JPG/320px-Masala_Chai.JPG")
    assert isinstance(result, dict)
    assert result["type"] == "Entity"
    assert result["text"] == "Chai"
    assert isinstance(result["properties"], dict)
    assert result["properties"]["Discontinued"]["basicValue"] is False
    assert result["properties"]["Image"]["address"].startswith("http")

def test_entity_type_discontinued():
    result = entity_type("Old Product", True, "https://example.com/old_product.jpg")
    assert isinstance(result, dict)
    assert result["text"] == "Old Product"
    assert result["properties"]["Discontinued"]["basicValue"] is True

def test_entity_type_invalid_text():
    result = entity_type(123, False, "https://example.com/img.jpg")
    assert isinstance(result, str) and "Error" in result

def test_entity_type_invalid_discontinued():
    result = entity_type("Chai", "no", "https://example.com/img.jpg")
    assert isinstance(result, str) and "Error" in result

def test_entity_type_invalid_image_url():
    result = entity_type("Chai", False, "not_a_url")
    assert isinstance(result, str) and "Error" in result

ipytest.run()

In [ ]:
import gradio as gr

examples = [
    ["Chai", False, "https://upload.wikimedia.org/wikipedia/commons/thumb/0/04/Masala_Chai.JPG/320px-Masala_Chai.JPG"],
    ["Old Product", True, "https://example.com/old_product.jpg"]
]

demo = gr.Interface(
    fn=entity_type,
    inputs=[
        gr.Textbox(label="Text", value="Chai"),
        gr.Checkbox(label="Discontinued", value=False),
        gr.Textbox(label="Image URL", value="https://upload.wikimedia.org/wikipedia/commons/thumb/0/04/Masala_Chai.JPG/320px-Masala_Chai.JPG")
    ],
    outputs="json",
    examples=examples,
    description="Create an Excel entity data type dictionary.",
    flagging_mode='never'
)
demo.launch()